## Import data

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, date


#Assign filenames
fullfile = 'Data/NM_2016_17export.csv'
ndfile = 'Data/2017 PMP Data Final No Duplicates.xlsx'
VDCfile = 'Data/Federal_VDC_Relation_web_clean.xlsx'
popfile = 'Data/nepal_population_household_ward-2011.csv'


#import dataframes

fulldf = pd.read_csv(fullfile)
ND17df = pd.read_excel(ndfile)
VDC_df = pd.read_excel(VDCfile,sheetname=1)
wardpop_df = pd.read_csv(popfile)

#Convert the incident date column into a datetime object for both date frames
fulldf['INCIDENT DATE'] = pd.to_datetime(fulldf['INCIDENT DATE'])

datelist = ND17df['INCIDENT DATE'].tolist() # convert excel time delta into datetimes
datelist = [pd.to_datetime('1899-12-30') + pd.to_timedelta(int(date),unit='D') for date in datelist]
ND17df['INCIDENT DATE'] = datelist

#let's round of the times
fulldf['INCIDENT DATE'] = fulldf['INCIDENT DATE'].dt.normalize()
ND17df['INCIDENT DATE'] = ND17df['INCIDENT DATE'].dt.normalize()

#index dataframes by incident number - apparently could have done this within read.csv function
def incident_index(df):
    df.index = df['#']
    del df['#']
    df.index.name = "INCIDENT NUMBER"
    return df

fulldf = incident_index(fulldf)
ND17df = incident_index(ND17df)

#Sort the dfs by index
fulldf = fulldf.sort_index(ascending = False)
ND17df = ND17df.sort_index(ascending = False)



## Initial data cleaning steps
* Remove unnecessary columns

* Convert yes no values to 1's and zeros

* Unpack numerical codes into columns?



To remove unnecissary columns:

The Nepal Monitor dataset, contains two general categories of data:
 * All human rights incidents
 * Violent incidents. 

The first type is coded for all human rights incidents, and the second is coded for a subset of violence and political contestation related incidents. If mapping personal determine an incident is violence related, they will indicate this and a second form will appear.  Fields for these forms will remain blank in the dataset those not violence related.





In [2]:
#View list of columns
fulldf.columns

Index(['INCIDENT TITLE', 'INCIDENT DATE', 'LOCATION', 'DESCRIPTION',
       'CATEGORY', 'LATITUDE', 'LONGITUDE', 'HLCIT CODE', 'LOCATION ACCURACY',
       'Country', 'Development Region', 'Zone', 'District', 'Municipality/VDC',
       'Ward', 'Country(New)', 'Development Region(New)', 'Zone(New)',
       'District(New)', 'Municipality/VDC(New)', 'Ward(New)', 'SOURCE',
       'SOURCE TYPE', 'Peace Monitoring - General Information',
       'Event Date - Peace Monitoring',
       '<a href="http://www.geonames.org/advanced-search.html?q=&country=NP&featureClass=&continentCode=&fuzzy=0.6" target="_blank">Search GeoNames</a>',
       'Duration - only applicable to non-violent forms', 'Number of Injuries',
       'Violent vs non-violent', 'One-sided vs two-sided', 'Number of Deaths',
       'Collective vs interpersonal', 'Forms and Causes',
       'Event form - primary', 'Event form - secondary', 'Weapon', 'Weapon 2',
       'Cause - primary', 'Cause - secondary', 'Crosscutting issues',
     

There is a spacer between the two forms in the website, which appears as an artifact in the dataset
This one happens to link to a useful geocoding search for mapping personel
<a href="http://www.geonames.org/advanced-search.html?q=&country=NP&featureClass=&continentCode=&fuzzy=0.6" target="_blank">Search GeoNames</a>. Let's filter for all null columns

In [3]:
def NaNcolumns(df): 
    '''Filter for null columns'''
    blankslug = [print(item) for item in df.columns[df.isnull().all() == True]]

NaNcolumns(fulldf)

Peace Monitoring - General Information
<a href="http://www.geonames.org/advanced-search.html?q=&country=NP&featureClass=&continentCode=&fuzzy=0.6" target="_blank">Search GeoNames</a>
Forms and Causes
Actor 1 - Perpetrator
Actor 2 - Perpetrator 2 or Target
Impacts
FIRST NAME
LAST NAME
EMAIL


As indicated by .isnull().all several additional spacers are present:
* Peace Monitoring - General Information
* Forms and Causes
* Actor 1 - Perpetrator
* Actor 2 - Perpetrator 2 or target
* Impacts

These could be useful for interpreting or slicing different sections of the dataset. The others are part of the mapping process and can be removed. 

Development regions and zones are not different in the new federal structures, and are redundant. 'Country' in both types should be unnecissary, as it is a Nepal dataset.

In [4]:
#Drop unnecissary columns
droplist = ['<a href="http://www.geonames.org/advanced-search.html?q=&country=NP&featureClass=&continentCode=&fuzzy=0.6" target="_blank">Search GeoNames</a>','FIRST NAME','LAST NAME','EMAIL', 'Country','Country(New)','Development Region(New)','Zone(New)']
fulldf.drop(droplist, axis=1, inplace=True)

#verify drops
NaNcolumns(fulldf)

Peace Monitoring - General Information
Forms and Causes
Actor 1 - Perpetrator
Actor 2 - Perpetrator 2 or Target
Impacts


* Covert yes/no values to ones and zeros 

In [5]:
#Look at the first item. The only columns that look like yes/no values are 'APPROVED' and 'VERIFIED'
fulldf.iloc[1]

#check for all columns that contain a variation of yes or no. 
def yes_no(df):
    for column in df.columns.tolist():
        clm = df[column].tolist()
        if ('YES' in clm) or ('NO' in clm) or ('yes' in clm) or ('no' in clm):
            print(column)

        
yes_no(fulldf)


APPROVED
VERIFIED


In [6]:
#As this is a public export, 'APPROVED' is 100% 'YES' values and can be dropped. 
fulldf.drop('APPROVED', axis=1, inplace=True)

In [7]:
#replace YES NO with 1 and 0
fulldf['VERIFIED'] = fulldf.VERIFIED.map(dict(YES=1, NO=0))
fulldf['VERIFIED'].sum()

35

In [8]:
#Investigate flatening data with pivot.
PMdf = fulldf.loc[:,'Peace Monitoring - General Information':'Impacts']
PMdf.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 7204 entries, 17685 to 8736
Data columns (total 28 columns):
Peace Monitoring - General Information             0 non-null float64
Event Date - Peace Monitoring                      4777 non-null object
Duration - only applicable to non-violent forms    298 non-null object
Number of Injuries                                 235 non-null float64
Violent vs non-violent                             4777 non-null object
One-sided vs two-sided                             404 non-null float64
Number of Deaths                                   3465 non-null object
Collective vs interpersonal                        4774 non-null object
Forms and Causes                                   0 non-null float64
Event form - primary                               7204 non-null object
Event form - secondary                             7204 non-null object
Weapon                                             7204 non-null object
Weapon 2                      

In [9]:
#find number of unique categories to flatten. 
PMunq = pd.Series(index=PMdf.columns.tolist())
for col in PMdf: 
    PMunq[col] = PMdf[col].nunique()

PMunq

Peace Monitoring - General Information               0.0
Event Date - Peace Monitoring                      564.0
Duration - only applicable to non-violent forms      4.0
Number of Injuries                                  11.0
Violent vs non-violent                               2.0
One-sided vs two-sided                               7.0
Number of Deaths                                     2.0
Collective vs interpersonal                          4.0
Forms and Causes                                     0.0
Event form - primary                                33.0
Event form - secondary                              29.0
Weapon                                              10.0
Weapon 2                                             9.0
Cause - primary                                     54.0
Cause - secondary                                   47.0
Crosscutting issues                                 19.0
Actor 1 - Perpetrator                                0.0
Actor 1 - Number of People     

In [10]:
set(PMdf['Event form - primary'])

{' ()',
 'Non-violent - Arrest (politically significant) (28)',
 'Non-violent - Banishment/shunning (32)',
 'Non-violent - Curfew/prohibitory order issues by the state (27)',
 'Non-violent - Demonstration (21)',
 'Non-violent - Extortion (29)',
 'Non-violent - Gherau/Dharna (sit-in)/padlocking (23)',
 'Non-violent - Other forms of non-violent political protest (24)',
 'Non-violent - Program Interference/Obstruction (241)',
 'Non-violent - Public humiliation (31)',
 'Non-violent - Raid/sweeping (26)',
 'Non-violent - Road block (25)',
 'Non-violent - Strike/Bandh (22)',
 'Non-violent - Threat/intimidation (30)',
 'Violent - Abduction (16)',
 'Violent - Arson (15)',
 'Violent - Assault (large group) (06)',
 'Violent - Assault (small group) (10)',
 'Violent - Fight (12)',
 'Violent - Group Clash (02)',
 'Violent - Infanticide (191)',
 'Violent - Murder/attempted Murder (19)',
 'Violent - Other violent form (0)',
 'Violent - Remote violence (08)',
 'Violent - Robbery (17)',
 'Violent - Sex

In [11]:
#convert total killed from string to float or integer
#discovered an errant dtype in a column while trying another project. - better to clean it here than there
print(fulldf['Total Killed'].dtype)
print(fulldf['Female Killed'].dtype)
#for item in fulldf['Total Killed']:
    #if item == '0-' 
    #if type(item) == str:
        #item = int(item)
print(fulldf['Total Killed'].loc[fulldf['Total Killed'] == '0-'])
fulldf['Total Killed'][17318] = '0'
fulldf['Total Killed'] = fulldf['Total Killed'].astype('float64')
print(fulldf['Total Killed'])

object
float64
INCIDENT NUMBER
17318    0-
Name: Total Killed, dtype: object
INCIDENT NUMBER
17685    0.0
17684    0.0
17675    0.0
17670    0.0
17668    0.0
17665    0.0
17662    1.0
17657    0.0
17651    0.0
17629    0.0
17622    0.0
17609    0.0
17608    0.0
17606    0.0
17600    0.0
17599    NaN
17597    0.0
17593    NaN
17591    NaN
17587    0.0
17586    0.0
17584    0.0
17583    0.0
17582    0.0
17580    0.0
17562    0.0
17550    NaN
17541    0.0
17532    NaN
17523    0.0
        ... 
8771     NaN
8770     NaN
8769     NaN
8767     NaN
8766     NaN
8765     NaN
8764     NaN
8763     NaN
8762     NaN
8759     NaN
8758     NaN
8756     NaN
8755     NaN
8753     NaN
8752     NaN
8750     NaN
8749     NaN
8748     NaN
8747     NaN
8746     NaN
8745     NaN
8744     NaN
8743     NaN
8742     NaN
8741     NaN
8740     NaN
8739     NaN
8738     NaN
8737     NaN
8736     NaN
Name: Total Killed, Length: 7204, dtype: float64


/home/nhorning/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


While it could useful in diffrent analysis contexts to flatten out these rows into columns, the "Event Form" rows themselves have 33 unique values with the potential to become their own column, so doing these for the entire set would make it unwieldy. As one row = one observation (incident) in the Nepal Monitor dataset, the dataset is already tidy.

## Merge in new federal structures

The federal structures have changed since the NepalMonior system was modified to accomodate them.  Merge those in from an RCO dataset. Possibly use this same function for adding them into population by ward data set and reposting to HDX. 




In [12]:
#Verify there are no duplicate incident numbers in the index 
fulldf.index.nunique() == len(fulldf.index)

True

In [13]:
#check VDC.
VDC_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7094 entries, 0 to 7093
Data columns (total 13 columns):
Province                  7094 non-null int64
District                  7094 non-null object
Pcode                     7094 non-null object
New Local unit English    7094 non-null object
Type                      7094 non-null object
Population                753 non-null float64
Area                      768 non-null object
Ward                      7094 non-null int64
VDC_Code                  7093 non-null object
Old VDC                   7093 non-null object
Old Ward                  7094 non-null object
Relationship Category     7092 non-null float64
Remarks                   65 non-null object
dtypes: float64(2), int64(2), object(9)
memory usage: 720.6+ KB


In [14]:
VDC_df[VDC_df['Old Ward'].isnull() == True]

,Province,District,Pcode,New Local unit English,Type,Population,Area,Ward,VDC_Code,Old VDC,Old Ward,Relationship Category,Remarks


In [15]:
#clean and break out wards in vdc.df
import re

def hyphen_range(s):
    """ Takes a range in form of "a-b" and generate a list of numbers between a and b inclusive.
    Also accepts comma separated ranges like "a-b,c-d,f" will build a list which will include
    Numbers from a to b, a to d and f
     code/recipes/Python/577279_Generate_List_Numbers_Hyphenated_commsepareted/recipe-577279.py """
    try:
        s=str(s)
        s="".join(s.split())#removes white space
        if s[-1] == ',': #removes trailing comma
            s = s[:-1]
        s = s.replace('.',',')#replaces erronious . with ,
        s = s.replace(',,',',')#removes double comma
        s = re.sub('(?<=\D)-|-(?=\D)|(?<=^)-|-(?=$)', '', s)
        r=set()
        for x in s.split(','):
            t=x.split('-')
            if len(t) not in [1,2]: raise SyntaxError("hash_range is given its arguement as "+s+" which seems not correctly formated.")
            r.add(int(t[0])) if len(t)==1 else r.update(set(range(int(t[0]),int(t[1])+1)))
        l=list(r)
        l.sort()
        #print(l)
        #l=",".join(l(x) for x in l)
        #l=l[0]
        return l 
    except:
        #print(s)
        return s

oldward_df = VDC_df #.loc[VDC_df['Old Ward'].str.find('-') <= 0]
#oldward_df['Old Ward'] = oldward_df['Old Ward'].apply(str)
#oldward_df['Old Ward'] = oldward_df['Old Ward'].str.replace('.',',')
#VDC_df['Old Ward'] = VDC_df['Old Ward'].str.replace('nan','0')

#Clean up dirty district names discovered in plotting
oldward_df['District'] = oldward_df['District'].str.replace('lamjung','Lamjung')
def getgoodbadlist(df):
    series = df['District'].dropna()
    somelist = list(set(series.tolist()))
    somelist = sorted(somelist)
    return somelist

badlist = getgoodbadlist(oldward_df)
goodlist = getgoodbadlist(fulldf)
for num in range(len(goodlist)):
    #print(badlist[num],goodlist[num])
    oldward_df['District'] = oldward_df['District'].str.replace(badlist[num],goodlist[num])

wardlist = oldward_df['Old Ward'].apply(hyphen_range)
oldward_df['Old Ward list'] = wardlist
oldward_df.head() #looks like we still have about 69 null values for old wards. 


,Province,District,Pcode,New Local unit English,Type,Population,Area,Ward,VDC_Code,Old VDC,Old Ward,Relationship Category,Remarks,Old Ward list
0,7,Achham,524 7 69 4 003,Bannigadhi Jayagadh,Rural Municipality,17359.0,58.26,1,524 5 13 69 5 037,Gajara,1-9,0.0,NaN,"[1, 2, 3, 4, 5, 6, 7, 8, 9]"
1,7,Achham,524 7 69 4 003,Bannigadhi Jayagadh,Rural Municipality,NaN,NaN,2,524 5 13 69 5 038,Janalikot,1-9,0.0,NaN,"[1, 2, 3, 4, 5, 6, 7, 8, 9]"
2,7,Achham,524 7 69 4 003,Bannigadhi Jayagadh,Rural Municipality,NaN,NaN,3,524 5 13 69 5 036,Baradadevi,1-9,0.0,NaN,"[1, 2, 3, 4, 5, 6, 7, 8, 9]"
3,7,Achham,524 7 69 4 003,Bannigadhi Jayagadh,Rural Municipality,NaN,NaN,4,524 5 13 69 5 040,Timilsain,1-9,0.0,NaN,"[1, 2, 3, 4, 5, 6, 7, 8, 9]"
4,7,Achham,524 7 69 4 003,Bannigadhi Jayagadh,Rural Municipality,NaN,NaN,5,524 5 13 69 5 039,Kalika,1-9,0.0,NaN,"[1, 2, 3, 4, 5, 6, 7, 8, 9]"


In [16]:
#oldward_df[oldward_df['Old Ward'].isnull() == True]
oldward_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7094 entries, 0 to 7093
Data columns (total 14 columns):
Province                  7094 non-null int64
District                  7094 non-null object
Pcode                     7094 non-null object
New Local unit English    7094 non-null object
Type                      7094 non-null object
Population                753 non-null float64
Area                      768 non-null object
Ward                      7094 non-null int64
VDC_Code                  7093 non-null object
Old VDC                   7093 non-null object
Old Ward                  7094 non-null object
Relationship Category     7092 non-null float64
Remarks                   65 non-null object
Old Ward list             7094 non-null object
dtypes: float64(2), int64(2), object(10)
memory usage: 776.0+ KB


In [17]:
#create multi-index population chart.
provincemerge = oldward_df[['Province','District','Population']]

#provincemerge.dropna(axis=0,inplace=True)
pop_p_d = provincemerge.groupby(['Province','District']).sum()
pop_p_d

Population
Province District                  
1        Bhojpur           180889.0
         Dhankuta          161398.0
         Ilam              287916.0
         Jhapa             808324.0
         Khotang           205225.0
         Morang            960876.0
         Okhaldhunga       146832.0
         Panchthar         191817.0
         Sankhuwasabha     158222.0
         Solukhumbu        105119.0
         Sunsari           753328.0
         Taplejung         127461.0
         Terhathum         100869.0
         Udayapur          315429.0
2        Bara              690837.0
         Dhanusa           742784.0
         Mahottari         637139.0
         Parsa             596086.0
         Rautahat          689915.0
         Saptari           627790.0
         Sarlahi           769330.0
         Siraha            636361.0
3        Bhaktapur         298684.0
         Chitawan          567988.0
         Dhading           336424.0
         Dolakha           185496.0
         Kathmandu        1699288.0
         Kavrepalanchok    375221.0
         Lalitpur          457606.0
         Makwanpur         419377.0
...                             ...
4        Tanahu            320547.0
5        Arghakhanchi      196896.0
         Banke             485164.0
         Bardiya           423656.0
         Dang              548140.0
         Gulmi             279008.0
         Kapilbastu        569844.0
         Nawalparasi       631922.0
         Palpa             258893.0
         Pyuthan           226796.0
         Rolpa             221177.0
         Rupandehi         874606.0
6        Dailekh           260855.0
         Dolpa              36128.0
         Humla              49973.0
         Jajarkot          170113.0
         Jumla             108066.0
         Kalikot           136587.0
         Mugu               54828.0
         Salyan            241716.0
         Surkhet           343318.0
7        Achham            256188.0
         Baitadi           250225.0
         Bajhang           194701.0
         Bajura            134154.0
         Dadeldhura        131004.0
         Darchula          132484.0
         Doti              207070.0
         Kailali           766659.0
         Kanchanpur        448503.0

[75 rows x 1 columns]

In [18]:
#save it for later
pop_p_d.to_csv('Data/District_Province_Population')

In [19]:
#merge the palicoldward_dfdfrd dataframes, then filter out where the wards don't match the ward list
fedf = pd.merge(wardpop_df, oldward_df, left_on='HLCIT_CODE_VDC', right_on='VDC_Code', how='outer')
fedf

,DISTRICT,V.D.C./MUNICIPALITY,dv_join,WARD NUMBER,HOUSEHOLD,POPULATION,CBS CODE,P_CODE_DIST,HLCIT_CODE_DIST,P_CODE_VDC,...,Type,Population,Area,Ward,VDC_Code,Old VDC,Old Ward,Relationship Category,Remarks,Old Ward list
0,Achham,Babala,AchhamBabala,1.0,19.0,90.0,690001.0,F-SET-69,524 5 13 69,F-SET-69-001,...,Municipality,NaN,NaN,13,524 5 13 69 5 008,Bawala,1-9,0.0,NaN,"[1, 2, 3, 4, 5, 6, 7, 8, 9]"
1,Achham,Babala,AchhamBabala,2.0,90.0,457.0,690001.0,F-SET-69,524 5 13 69,F-SET-69-001,...,Municipality,NaN,NaN,13,524 5 13 69 5 008,Bawala,1-9,0.0,NaN,"[1, 2, 3, 4, 5, 6, 7, 8, 9]"
2,Achham,Babala,AchhamBabala,3.0,111.0,596.0,690001.0,F-SET-69,524 5 13 69,F-SET-69-001,...,Municipality,NaN,NaN,13,524 5 13 69 5 008,Bawala,1-9,0.0,NaN,"[1, 2, 3, 4, 5, 6, 7, 8, 9]"
3,Achham,Babala,AchhamBabala,4.0,87.0,471.0,690001.0,F-SET-69,524 5 13 69,F-SET-69-001,...,Municipality,NaN,NaN,13,524 5 13 69 5 008,Bawala,1-9,0.0,NaN,"[1, 2, 3, 4, 5, 6, 7, 8, 9]"
4,Achham,Babala,AchhamBabala,5.0,31.0,161.0,690001.0,F-SET-69,524 5 13 69,F-SET-69-001,...,Municipality,NaN,NaN,13,524 5 13 69 5 008,Bawala,1-9,0.0,NaN,"[1, 2, 3, 4, 5, 6, 7, 8, 9]"
5,Achham,Babala,AchhamBabala,6.0,40.0,241.0,690001.0,F-SET-69,524 5 13 69,F-SET-69-001,...,Municipality,NaN,NaN,13,524 5 13 69 5 008,Bawala,1-9,0.0,NaN,"[1, 2, 3, 4, 5, 6, 7, 8, 9]"
6,Achham,Babala,AchhamBabala,7.0,36.0,182.0,690001.0,F-SET-69,524 5 13 69,F-SET-69-001,...,Municipality,NaN,NaN,13,524 5 13 69 5 008,Bawala,1-9,0.0,NaN,"[1, 2, 3, 4, 5, 6, 7, 8, 9]"
7,Achham,Babala,AchhamBabala,8.0,89.0,430.0,690001.0,F-SET-69,524 5 13 69,F-SET-69-001,...,Municipality,NaN,NaN,13,524 5 13 69 5 008,Bawala,1-9,0.0,NaN,"[1, 2, 3, 4, 5, 6, 7, 8, 9]"
8,Achham,Babala,AchhamBabala,9.0,36.0,185.0,690001.0,F-SET-69,524 5 13 69,F-SET-69-001,...,Municipality,NaN,NaN,13,524 5 13 69 5 008,Bawala,1-9,0.0,NaN,"[1, 2, 3, 4, 5, 6, 7, 8, 9]"
9,Achham,Baijinath,AchhamBaijinath,1.0,47.0,219.0,690002.0,F-SET-69,524 5 13 69,F-SET-69-002,...,Municipality,33788.0,166.71,1,524 5 13 69 5 001,Baijanath,1,1.0,NaN,[1]


In [20]:
def inwardlst(ward, lst):
    
    try:
        for item in lst:
            if int(ward) == int(item):
                return True
        #print (ward, lst, 'False!')
        return False
    except:
        print (ward, lst, 'exception!')
        return True

inwardlist = [inwardlst(fedf['WARD NUMBER'][x], fedf['Old Ward list'][x]) for x in fedf.index]
fedf = fedf.loc[inwardlist]
fedf

nan [1, 2, 5] exception!
nan [12, 13] exception!


,DISTRICT,V.D.C./MUNICIPALITY,dv_join,WARD NUMBER,HOUSEHOLD,POPULATION,CBS CODE,P_CODE_DIST,HLCIT_CODE_DIST,P_CODE_VDC,...,Type,Population,Area,Ward,VDC_Code,Old VDC,Old Ward,Relationship Category,Remarks,Old Ward list
0,Achham,Babala,AchhamBabala,1.0,19.0,90.0,690001.0,F-SET-69,524 5 13 69,F-SET-69-001,...,Municipality,NaN,NaN,13,524 5 13 69 5 008,Bawala,1-9,0.0,NaN,"[1, 2, 3, 4, 5, 6, 7, 8, 9]"
1,Achham,Babala,AchhamBabala,2.0,90.0,457.0,690001.0,F-SET-69,524 5 13 69,F-SET-69-001,...,Municipality,NaN,NaN,13,524 5 13 69 5 008,Bawala,1-9,0.0,NaN,"[1, 2, 3, 4, 5, 6, 7, 8, 9]"
2,Achham,Babala,AchhamBabala,3.0,111.0,596.0,690001.0,F-SET-69,524 5 13 69,F-SET-69-001,...,Municipality,NaN,NaN,13,524 5 13 69 5 008,Bawala,1-9,0.0,NaN,"[1, 2, 3, 4, 5, 6, 7, 8, 9]"
3,Achham,Babala,AchhamBabala,4.0,87.0,471.0,690001.0,F-SET-69,524 5 13 69,F-SET-69-001,...,Municipality,NaN,NaN,13,524 5 13 69 5 008,Bawala,1-9,0.0,NaN,"[1, 2, 3, 4, 5, 6, 7, 8, 9]"
4,Achham,Babala,AchhamBabala,5.0,31.0,161.0,690001.0,F-SET-69,524 5 13 69,F-SET-69-001,...,Municipality,NaN,NaN,13,524 5 13 69 5 008,Bawala,1-9,0.0,NaN,"[1, 2, 3, 4, 5, 6, 7, 8, 9]"
5,Achham,Babala,AchhamBabala,6.0,40.0,241.0,690001.0,F-SET-69,524 5 13 69,F-SET-69-001,...,Municipality,NaN,NaN,13,524 5 13 69 5 008,Bawala,1-9,0.0,NaN,"[1, 2, 3, 4, 5, 6, 7, 8, 9]"
6,Achham,Babala,AchhamBabala,7.0,36.0,182.0,690001.0,F-SET-69,524 5 13 69,F-SET-69-001,...,Municipality,NaN,NaN,13,524 5 13 69 5 008,Bawala,1-9,0.0,NaN,"[1, 2, 3, 4, 5, 6, 7, 8, 9]"
7,Achham,Babala,AchhamBabala,8.0,89.0,430.0,690001.0,F-SET-69,524 5 13 69,F-SET-69-001,...,Municipality,NaN,NaN,13,524 5 13 69 5 008,Bawala,1-9,0.0,NaN,"[1, 2, 3, 4, 5, 6, 7, 8, 9]"
8,Achham,Babala,AchhamBabala,9.0,36.0,185.0,690001.0,F-SET-69,524 5 13 69,F-SET-69-001,...,Municipality,NaN,NaN,13,524 5 13 69 5 008,Bawala,1-9,0.0,NaN,"[1, 2, 3, 4, 5, 6, 7, 8, 9]"
9,Achham,Baijinath,AchhamBaijinath,1.0,47.0,219.0,690002.0,F-SET-69,524 5 13 69,F-SET-69-002,...,Municipality,33788.0,166.71,1,524 5 13 69 5 001,Baijanath,1,1.0,NaN,[1]


In [21]:
wardpop_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36020 entries, 0 to 36019
Data columns (total 14 columns):
DISTRICT                 36020 non-null object
V.D.C./MUNICIPALITY      36020 non-null object
dv_join                  36020 non-null object
WARD NUMBER              36020 non-null int64
HOUSEHOLD                36020 non-null int64
POPULATION               36020 non-null int64
CBS CODE                 36020 non-null int64
P_CODE_DIST              36020 non-null object
HLCIT_CODE_DIST          36020 non-null object
P_CODE_VDC               36020 non-null object
HLCIT_CODE_VDC           36020 non-null object
P_CODE_WARD              36020 non-null object
HLCIT_CODE_WARD          36020 non-null object
elevation_above_1500m    36020 non-null int64
dtypes: int64(5), object(9)
memory usage: 3.8+ MB


In [22]:
#fed = fulldf.join(provincemerge, on='District', how='left')

In [23]:
#Merge accurate federal structures into df

#check if a given ward num is in the old ward list
def inwardlst(ward, lst):
    try:
        for item in lst:
            if int(ward) == int(item):
                return True
        return False
    except:
        print (ward, lst, 'False!')
        return False



VDCmerge = oldward_df[['Pcode','Province','New Local unit English','Ward','Old Ward','VDC_Code','Old Ward list']]

#VDCmerge = VDCmerge.set_index('VDC_Code', drop=True)
VDCmerge.columns = ['New Pcode','Province','New local unit','New Ward','Old Ward','VDC_Code','Old Ward list']
fedf = pd.DataFrame()
wardcodes = wardpop_df['HLCIT_CODE_WARD']
for HLCIT in wardcodes:
    #HLCIT = wardpop_df.loc[idx]['HLCIT_CODE_WARD']
    #length = 17
    #HLCIT = str(HLCIT)
    matchdf = VDCmerge.loc[VDCmerge['VDC_Code'] == HLCIT[:17]]
    ward = int(HLCIT[17:])
    oldwards = matchdf['Old Ward list']
    inoldward = [inwardlst(ward,lst) for lst in oldwards]
    matchdf = matchdf.loc[inoldward]
    matchdf['HLCIT_CODE_WARD'] = HLCIT
        #print(inoldward)
    fedf = pd.concat([fedf,matchdf])
    #fedf[HLCIT] = matchdf[1:]
    #for columname in matchdf.columns.tolist()[:-1]:
        #column = matchdf[columname].tolist()
        #if len(set(column)) == 1:
            #fedf.loc[idx][columname] = column[0]
    #progress = round(len(wardcodes[:HLCIT])/len(wardcodes),2)
    print(HLCIT)
#fedf
#inoldward
#matchdf.head()
#problem is that the left frame only stores the relivant numbers of the HLCIT code. 
#A join is not possible, #match by matching len to slice slicing, appendin the first result to a new dataframe
#with the same index, and then concating by index

#create df with all the matches, and then pull in the ones where all the values are the same - set length = 1
#fedf.head()

524 5 13 69 5 008 1
524 5 13 69 5 008 2
524 5 13 69 5 008 3
524 5 13 69 5 008 4
524 5 13 69 5 008 5
524 5 13 69 5 008 6
524 5 13 69 5 008 7
524 5 13 69 5 008 8
524 5 13 69 5 008 9
524 5 13 69 5 001 1
524 5 13 69 5 001 2
524 5 13 69 5 001 3
524 5 13 69 5 001 4
524 5 13 69 5 001 5
524 5 13 69 5 001 6
524 5 13 69 5 001 7
524 5 13 69 5 001 8
524 5 13 69 5 001 9
524 5 13 69 5 028 1
524 5 13 69 5 028 2
524 5 13 69 5 028 3
524 5 13 69 5 028 4
524 5 13 69 5 028 5
524 5 13 69 5 028 6
524 5 13 69 5 028 7
524 5 13 69 5 028 8
524 5 13 69 5 028 9
524 5 13 69 5 036 1
524 5 13 69 5 036 2
524 5 13 69 5 036 3
524 5 13 69 5 036 4
524 5 13 69 5 036 5
524 5 13 69 5 036 6
524 5 13 69 5 036 7
524 5 13 69 5 036 8
524 5 13 69 5 036 9
524 5 13 69 5 049 1
524 5 13 69 5 049 2
524 5 13 69 5 049 3
524 5 13 69 5 049 4
524 5 13 69 5 049 5
524 5 13 69 5 049 6
524 5 13 69 5 049 7
524 5 13 69 5 049 8
524 5 13 69 5 049 9
524 5 13 69 5 041 1
524 5 13 69 5 041 2
524 5 13 69 5 041 3
524 5 13 69 5 041 4
524 5 13 69 5 041 5


524 5 13 69 5 034 3
524 5 13 69 5 034 4
524 5 13 69 5 034 5
524 5 13 69 5 034 6
524 5 13 69 5 034 7
524 5 13 69 5 034 8
524 5 13 69 5 034 9
524 5 13 69 5 004 1
524 5 13 69 5 004 2
524 5 13 69 5 004 3
524 5 13 69 5 004 4
524 5 13 69 5 004 5
524 5 13 69 5 004 6
524 5 13 69 5 004 7
524 5 13 69 5 004 8
524 5 13 69 5 004 9
524 5 13 69 5 075 1
524 5 13 69 5 075 2
524 5 13 69 5 075 3
524 5 13 69 5 075 4
524 5 13 69 5 075 5
524 5 13 69 5 075 6
524 5 13 69 5 075 7
524 5 13 69 5 075 8
524 5 13 69 5 075 9
524 5 13 69 5 019 1
524 5 13 69 5 019 2
524 5 13 69 5 019 3
524 5 13 69 5 019 4
524 5 13 69 5 019 5
524 5 13 69 5 019 6
524 5 13 69 5 019 7
524 5 13 69 5 019 8
524 5 13 69 5 019 9
524 5 13 69 5 064 1
524 5 13 69 5 064 2
524 5 13 69 5 064 3
524 5 13 69 5 064 4
524 5 13 69 5 064 5
524 5 13 69 5 064 6
524 5 13 69 5 064 7
524 5 13 69 5 064 8
524 5 13 69 5 064 9
524 5 13 69 5 021 1
524 5 13 69 5 021 2
524 5 13 69 5 021 3
524 5 13 69 5 021 4
524 5 13 69 5 021 5
524 5 13 69 5 021 6
524 5 13 69 5 021 7


524 3 09 50 5 010 1
524 3 09 50 5 010 2
524 3 09 50 5 010 3
524 3 09 50 5 010 4
524 3 09 50 5 010 5
524 3 09 50 5 010 6
524 3 09 50 5 010 7
524 3 09 50 5 010 8
524 3 09 50 5 010 9
524 3 09 50 5 036 1
524 3 09 50 5 036 2
524 3 09 50 5 036 3
524 3 09 50 5 036 4
524 3 09 50 5 036 5
524 3 09 50 5 036 6
524 3 09 50 5 036 7
524 3 09 50 5 036 8
524 3 09 50 5 036 9
524 3 09 50 5 035 1
524 3 09 50 5 035 2
524 3 09 50 5 035 3
524 3 09 50 5 035 4
524 3 09 50 5 035 5
524 3 09 50 5 035 6
524 3 09 50 5 035 7
524 3 09 50 5 035 8
524 3 09 50 5 035 9
524 3 09 50 5 006 1
524 3 09 50 5 006 2
524 3 09 50 5 006 3
524 3 09 50 5 006 4
524 3 09 50 5 006 5
524 3 09 50 5 006 6
524 3 09 50 5 006 7
524 3 09 50 5 006 8
524 3 09 50 5 006 9
524 3 09 50 5 003 1
524 3 09 50 5 003 2
524 3 09 50 5 003 3
524 3 09 50 5 003 4
524 3 09 50 5 003 5
524 3 09 50 5 003 6
524 3 09 50 5 003 7
524 3 09 50 5 003 8
524 3 09 50 5 003 9
524 3 09 50 5 013 1
524 3 09 50 5 013 2
524 3 09 50 5 013 3
524 3 09 50 5 013 4
524 3 09 50 5 013 5


524 3 08 43 5 035 1
524 3 08 43 5 035 2
524 3 08 43 5 035 3
524 3 08 43 5 035 4
524 3 08 43 5 035 5
524 3 08 43 5 035 6
524 3 08 43 5 035 7
524 3 08 43 5 035 8
524 3 08 43 5 035 9
524 3 08 43 5 041 1
524 3 08 43 5 041 2
524 3 08 43 5 041 3
524 3 08 43 5 041 4
524 3 08 43 5 041 5
524 3 08 43 5 041 6
524 3 08 43 5 041 7
524 3 08 43 5 041 8
524 3 08 43 5 041 9
524 3 08 43 5 016 1
524 3 08 43 5 016 2
524 3 08 43 5 016 3
524 3 08 43 5 016 4
524 3 08 43 5 016 5
524 3 08 43 5 016 6
524 3 08 43 5 016 7
524 3 08 43 5 016 8
524 3 08 43 5 016 9
524 3 08 43 5 020 1
524 3 08 43 5 020 2
524 3 08 43 5 020 3
524 3 08 43 5 020 4
524 3 08 43 5 020 5
524 3 08 43 5 020 6
524 3 08 43 5 020 7
524 3 08 43 5 020 8
524 3 08 43 5 020 9
524 3 08 43 5 017 1
524 3 08 43 5 017 2
524 3 08 43 5 017 3
524 3 08 43 5 017 4
524 3 08 43 5 017 5
524 3 08 43 5 017 6
524 3 08 43 5 017 7
524 3 08 43 5 017 8
524 3 08 43 5 017 9
524 3 08 43 5 036 1
524 3 08 43 5 036 2
524 3 08 43 5 036 3
524 3 08 43 5 036 4
524 3 08 43 5 036 5


524 5 14 74 3 001 3
524 5 14 74 3 001 4
524 5 14 74 3 001 5
524 5 14 74 3 001 6
524 5 14 74 3 001 7
524 5 14 74 3 001 8
524 5 14 74 3 001 9
524 5 14 74 3 001 10
524 5 14 74 3 001 11
524 5 14 74 3 001 12
524 5 14 74 3 001 13
524 5 14 74 5 039 1
524 5 14 74 5 039 2
524 5 14 74 5 039 3
524 5 14 74 5 039 4
524 5 14 74 5 039 5
524 5 14 74 5 039 6
524 5 14 74 5 039 7
524 5 14 74 5 039 8
524 5 14 74 5 039 9
524 5 14 74 5 045 1
524 5 14 74 5 045 2
524 5 14 74 5 045 3
524 5 14 74 5 045 4
524 5 14 74 5 045 5
524 5 14 74 5 045 6
524 5 14 74 5 045 7
524 5 14 74 5 045 8
524 5 14 74 5 045 9
524 5 14 74 5 047 1
524 5 14 74 5 047 2
524 5 14 74 5 047 3
524 5 14 74 5 047 4
524 5 14 74 5 047 5
524 5 14 74 5 047 6
524 5 14 74 5 047 7
524 5 14 74 5 047 8
524 5 14 74 5 047 9
524 5 14 74 5 044 1
524 5 14 74 5 044 2
524 5 14 74 5 044 3
524 5 14 74 5 044 4
524 5 14 74 5 044 5
524 5 14 74 5 044 6
524 5 14 74 5 044 7
524 5 14 74 5 044 8
524 5 14 74 5 044 9
524 5 14 74 5 022 1
524 5 14 74 5 022 2
524 5 14 74 5 02

...

524 3 07 41 5 016 9
524 3 07 41 5 058 1
524 3 07 41 5 058 2
524 3 07 41 5 058 3
524 3 07 41 5 058 4
524 3 07 41 5 058 5
524 3 07 41 5 058 6
524 3 07 41 5 058 7
524 3 07 41 5 058 8
524 3 07 41 5 058 9
524 3 07 41 3 001 1
524 3 07 41 3 001 2
524 3 07 41 3 001 3
524 3 07 41 3 001 4
524 3 07 41 3 001 5
524 3 07 41 3 001 6
524 3 07 41 3 001 7
524 3 07 41 3 001 8
524 3 07 41 3 001 9
524 3 07 41 3 001 10
524 3 07 41 3 001 11
524 3 07 41 3 001 12
524 3 07 41 3 001 13
524 3 07 41 5 015 1
524 3 07 41 5 015 2
524 3 07 41 5 015 3
524 3 07 41 5 015 4
524 3 07 41 5 015 5
524 3 07 41 5 015 6
524 3 07 41 5 015 7
524 3 07 41 5 015 8
524 3 07 41 5 015 9
524 3 07 41 5 022 1
524 3 07 41 5 022 2
524 3 07 41 5 022 3
524 3 07 41 5 022 4
524 3 07 41 5 022 5
524 3 07 41 5 022 6
524 3 07 41 5 022 7
524 3 07 41 5 022 8
524 3 07 41 5 022 9
524 3 07 41 5 046 1
524 3 07 41 5 046 2
524 3 07 41 5 046 3
524 3 07 41 5 046 4
524 3 07 41 5 046 5
524 3 07 41 5 046 6
524 3 07 41 5 046 7
524 3 07 41 5 046 8
524 3 07 41 5 04

524 3 07 40 5 019 1
524 3 07 40 5 019 2
524 3 07 40 5 019 3
524 3 07 40 5 019 4
524 3 07 40 5 019 5
524 3 07 40 5 019 6
524 3 07 40 5 019 7
524 3 07 40 5 019 8
524 3 07 40 5 019 9
524 3 07 40 5 044 1
524 3 07 40 5 044 2
524 3 07 40 5 044 3
524 3 07 40 5 044 4
524 3 07 40 5 044 5
524 3 07 40 5 044 6
524 3 07 40 5 044 7
524 3 07 40 5 044 8
524 3 07 40 5 044 9
524 3 07 40 5 036 1
524 3 07 40 5 036 2
524 3 07 40 5 036 3
524 3 07 40 5 036 4
524 3 07 40 5 036 5
524 3 07 40 5 036 6
524 3 07 40 5 036 7
524 3 07 40 5 036 8
524 3 07 40 5 036 9
524 3 07 40 5 029 1
524 3 07 40 5 029 2
524 3 07 40 5 029 3
524 3 07 40 5 029 4
524 3 07 40 5 029 5
524 3 07 40 5 029 6
524 3 07 40 5 029 7
524 3 07 40 5 029 8
524 3 07 40 5 029 9
524 3 07 40 5 021 1
524 3 07 40 5 021 2
524 3 07 40 5 021 3
524 3 07 40 5 021 4
524 3 07 40 5 021 5
524 3 07 40 5 021 6
524 3 07 40 5 021 7
524 3 07 40 5 021 8
524 3 07 40 5 021 9
524 3 07 40 5 031 1
524 3 07 40 5 031 2
524 3 07 40 5 031 3
524 3 07 40 5 031 4
524 3 07 40 5 031 5


524 1 01 01 5 031 1
524 1 01 01 5 031 2
524 1 01 01 5 031 3
524 1 01 01 5 031 4
524 1 01 01 5 031 5
524 1 01 01 5 031 6
524 1 01 01 5 031 7
524 1 01 01 5 031 8
524 1 01 01 5 031 9
524 1 01 01 5 049 1
524 1 01 01 5 049 2
524 1 01 01 5 049 4
524 1 01 01 5 049 5
524 1 01 01 5 049 6
524 1 01 01 5 049 7
524 1 01 01 5 049 8
524 1 01 01 5 049 9
524 1 01 01 5 042 1
524 1 01 01 5 042 2
524 1 01 01 5 042 3
524 1 01 01 5 042 4
524 1 01 01 5 042 5
524 1 01 01 5 042 6
524 1 01 01 5 042 7
524 1 01 01 5 042 8
524 1 01 01 5 042 9
524 1 01 01 5 008 1
524 1 01 01 5 008 2
524 1 01 01 5 008 3
524 1 01 01 5 008 4
524 1 01 01 5 008 5
524 1 01 01 5 008 6
524 1 01 01 5 008 7
524 1 01 01 5 008 8
524 1 01 01 5 008 9
524 1 01 01 5 035 1
524 1 01 01 5 035 2
524 1 01 01 5 035 3
524 1 01 01 5 035 4
524 1 01 01 5 035 5
524 1 01 01 5 035 6
524 1 01 01 5 035 7
524 1 01 01 5 035 8
524 1 01 01 5 035 9
524 1 01 01 5 013 1
524 1 01 01 5 013 2
524 1 01 01 5 013 3
524 1 01 01 5 013 4
524 1 01 01 5 013 5
524 1 01 01 5 013 6


524 1 02 08 5 031 1
524 1 02 08 5 031 2
524 1 02 08 5 031 3
524 1 02 08 5 031 4
524 1 02 08 5 031 5
524 1 02 08 5 031 6
524 1 02 08 5 031 7
524 1 02 08 5 031 8
524 1 02 08 5 031 9
524 1 02 08 5 003 1
524 1 02 08 5 003 2
524 1 02 08 5 003 3
524 1 02 08 5 003 4
524 1 02 08 5 003 5
524 1 02 08 5 003 6
524 1 02 08 5 003 7
524 1 02 08 5 003 8
524 1 02 08 5 003 9
524 1 02 08 5 029 1
524 1 02 08 5 029 2
524 1 02 08 5 029 3
524 1 02 08 5 029 4
524 1 02 08 5 029 5
524 1 02 08 5 029 6
524 1 02 08 5 029 7
524 1 02 08 5 029 8
524 1 02 08 5 029 9
524 1 02 08 5 022 1
524 1 02 08 5 022 2
524 1 02 08 5 022 3
524 1 02 08 5 022 4
524 1 02 08 5 022 5
524 1 02 08 5 022 6
524 1 02 08 5 022 7
524 1 02 08 5 022 8
524 1 02 08 5 022 9
524 1 02 08 5 032 1
524 1 02 08 5 032 2
524 1 02 08 5 032 3
524 1 02 08 5 032 4
524 1 02 08 5 032 5
524 1 02 08 5 032 6
524 1 02 08 5 032 7
524 1 02 08 5 032 8
524 1 02 08 5 032 9
524 1 02 08 5 020 1
524 1 02 08 5 020 2
524 1 02 08 5 020 3
524 1 02 08 5 020 4
524 1 02 08 5 020 5


524 1 03 14 5 008 2
524 1 03 14 5 008 3
524 1 03 14 5 008 4
524 1 03 14 5 008 5
524 1 03 14 5 008 6
524 1 03 14 5 008 7
524 1 03 14 5 008 8
524 1 03 14 5 008 9
524 1 03 14 5 038 1
524 1 03 14 5 038 2
524 1 03 14 5 038 3
524 1 03 14 5 038 4
524 1 03 14 5 038 5
524 1 03 14 5 038 6
524 1 03 14 5 038 7
524 1 03 14 5 038 8
524 1 03 14 5 038 9
524 1 03 14 5 012 1
524 1 03 14 5 012 2
524 1 03 14 5 012 3
524 1 03 14 5 012 4
524 1 03 14 5 012 5
524 1 03 14 5 012 6
524 1 03 14 5 012 7
524 1 03 14 5 012 8
524 1 03 14 5 012 9
524 1 03 14 5 033 1
524 1 03 14 5 033 2
524 1 03 14 5 033 3
524 1 03 14 5 033 4
524 1 03 14 5 033 5
524 1 03 14 5 033 6
524 1 03 14 5 033 7
524 1 03 14 5 033 8
524 1 03 14 5 033 9
524 1 03 14 5 039 1
524 1 03 14 5 039 2
524 1 03 14 5 039 3
524 1 03 14 5 039 4
524 1 03 14 5 039 5
524 1 03 14 5 039 6
524 1 03 14 5 039 7
524 1 03 14 5 039 8
524 1 03 14 5 039 9
524 1 03 14 5 013 1
524 1 03 14 5 013 2
524 1 03 14 5 013 3
524 1 03 14 5 013 4
524 1 03 14 5 013 5
524 1 03 14 5 013 6


In [26]:
fedf.head()

,DISTRICT,V.D.C./MUNICIPALITY,dv_join,WARD NUMBER,HOUSEHOLD,POPULATION,CBS CODE,P_CODE_DIST,HLCIT_CODE_DIST,P_CODE_VDC,...,P_CODE_WARD,HLCIT_CODE_WARD,elevation_above_1500m,New Pcode,Province,New local unit,New Ward,Old Ward,VDC_Code,Old Ward list
0,Udayapur,Mayankhu,UdayapurMayankhu,9,40,207,140045,E-SAG-14,524 1 03 14,E-SAG-14-045,...,E-SAG-14-045-09,524 1 03 14 5 025 9,0,524 1 14 3 001,1,Katari,13,"1-3, 5-9",524 1 03 14 5 025,"[1, 2, 3, 5, 6, 7, 8, 9]"


In [23]:
#find why various pcodes do not match - check if the whole row is null - this is simply too many. 
count = 0
for num in fedf.index:
    if fedf.loc[num].isnull().all():
        count += 1
        print(num, wardpop_df.loc[num]['HLCIT_CODE_WARD'], )
count

NameError: name 'fedf' is not defined

## Remove Duplicates

In [ ]:
#Verify there are no duplicate incident numbers in the index 
fulldf.index.nunique() == len(fulldf.index)

In [48]:
#Eyeball the difference between the no duplicates file and the strait export. 
fulldf.info() 
#ND17df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7204 entries, 17685 to 8736
Data columns (total 67 columns):
INCIDENT TITLE                                        7204 non-null object
INCIDENT DATE                                         7204 non-null datetime64[ns]
LOCATION                                              7204 non-null object
DESCRIPTION                                           7204 non-null object
CATEGORY                                              7203 non-null object
LATITUDE                                              7204 non-null float64
LONGITUDE                                             7204 non-null float64
HLCIT CODE                                            7204 non-null object
LOCATION ACCURACY                                     7204 non-null object
Development Region                                    7079 non-null object
Zone                                                  7075 non-null object
District                                              

In [ ]:
ND17df.head()

In [ ]:

#Ensure the dates match for any given item / find where they don't
#mask = fulldf.mask(fulldf['INCIDENT DATE'] != ND17df['INCIDENT DATE']) <-- won't work becasue index is of different length
def match_check(df1,df2,column_name):  #lets make it a function so we don't have to replicate it
    for num in df1.index:
        if (num in df2.index) and (df1.loc[num][column_name] != df2.loc[num][column_name]):
            return False
    return True

print('Dates Match =', match_check(fulldf,ND17df,'INCIDENT DATE'))



In [27]:
#compare 1 full df to duplicate df to see what has been removed
def makediff(df1,df2):
    df1['difflist'] = [num not in df2.index for num in df1.index] #create a column of bools in df1-true if they are not in df2's index
    diffdf = df1.loc[df1['difflist'] == True] # Diffdf is now a slice of what is in df1 but not df2
    del diffdf['difflist'] # remove the new column
    return diffdf

#Slice fulldf down to only 2017
full17df = fulldf.loc[fulldf['INCIDENT DATE'].dt.year == 2017]

#calculate difference
diffdropped = makediff(full17df, ND17df)  # These items were removed from the 2017 data to make the no duplicates file '
diff17df = makediff(ND17df, full17df)  # This should what was added into the "No duplicates" file 'ND17df'

statsdf = pd.DataFrame(index=['full17df','ND17df','diff17df','diffdropped'], columns = ['shape','comments'])
statsdf['comments'] = ['The original 2017 export','No duplicates file','Added to make no duplicates file','The duplicates - Removed to make ND17df']
statsdf.index.name = 'Dataframe'
statsdf['shape'] = [full17df.shape, ND17df.shape, diff17df.shape, diffdropped.shape]
statsdf.head()

/home/nhorning/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,shape,comments
Dataframe,,
full17df,"(4115, 68)",The original 2017 export
ND17df,"(3555, 70)",No duplicates file
diff17df,"(13, 69)",Added to make no duplicates file
diffdropped,"(573, 67)",The duplicates - Removed to make ND17df


In [28]:
diff17df #These were what was added to make ND17df. 
#They are all on the last day, besides the two from 2018. Probably should be in final file

,INCIDENT TITLE,INCIDENT DATE,LOCATION,DESCRIPTION,CATEGORY,LATITUDE,LONGITUDE,HLCIT CODE,LOCATION ACCURACY,Sushila Karki,...,Youth -25 years or below- abducted,Total Buildings,Severe Building Damage,Total Vehicles,Severe Vehicle Damage,Feedback,Election-related specific cause primary,QC,Election-related specific cause secondary,Verification Feedback
INCIDENT NUMBER,,,,,,,,,,,,,,,,,,,,,
17687,Janakpur Today: RJPN cadres demonstrated again...,2017-12-31,"Gaur, Rautahat, NARAYANI, Central Development ...","Alert: Dhanusha/Dec 31, Samsad Anil Jha biurud...","Protest / Demonstration, News Media, Other Ele...",26.774839,85.258531,524 2 06 32 3 001,Municipality/VDC,NaN,...,0.0,0,0,0,0,NaN,Intraparty competition,Checked by Field,NaN,NaN
17506,INSEC: Arrested on charge of child molestation...,2018-01-01,"Rajbiraj-10, Saptari, SAGARMATHA, Eastern Deve...","Alert: Saptari/Jan 2, 12 barshiya balika lai y...","Trusted Reports, INSEC, Nepal Police (respondi...",26.537218,86.750468,524 1 03 15 3 001 3,Ward,NaN,...,0.0,0,0,0,0,NaN,NaN,Checked by Admin,NaN,NaN
17495,eKantipur: Two workers found dead | à¤®à¥‹à¤¬à...,2017-12-31,"Pokhara Lekhnath, Kaski, GANDAKI, Western Deve...","Alert: Kaski/Jan 1, Mobile chore pachhi gaali-...","Killings, INJURED, DEATHS, Nepal Police (respo...",28.243930,83.948625,524 3 07 39 5 043,Municipality/VDC,NaN,...,0.0,0,0,0,0,NaN,NaN,Checked by Admin,NaN,NaN
17492,Annapurna: Locals of Dipayal protest demanding...,2017-12-31,"Dipayal Silgadi, Doti, SETI, Far-Western Devel...","Alert: Doti/Jan 1, Pradesh 7 ko rajdhani Dipay...","LOCAL COMMUNITIES, Protest / Demonstration, Ne...",29.282945,80.891709,524 5 13 70 3 001,Municipality/VDC,NaN,...,0.0,0,0,0,0,NaN,NaN,Checked by Field,NaN,NaN
17486,News: Bandh called protesting change of rural ...,2017-12-31,"Suwarnabati, Rolpa, RAPTI, Mid-Western Develop...","Alert: Rolpa/Dec 31, Suvarnawoti gaunpalika ko...","LOCAL COMMUNITIES, News Media, Strike, Inciden...",28.198278,82.761318,524 4 10 54 5 005,Municipality/VDC,NaN,...,0.0,0,0,0,0,NaN,NaN,Checked by Admin,NaN,NaN
17484,"Alert: Man found dead in Banglachuli, suspect ...",2017-12-31,"Ghorahi-6, Dang, RAPTI, Mid-Western Developmen...","Alert: Dang/Jan 1, Bangalachuli gaunpalika maa...","Killings, DEATHS, Nepal Police (responding), D...",27.996877,82.449305,524 4 10 56 5 021 2,Ward,NaN,...,0.0,0,0,0,0,NaN,NaN,Check,NaN,NaN
17482,News: Pressure rally organized demanding Birat...,2018-01-01,"Biratnagar, Morang, KOSHI, Eastern Development...","Alert: Parsa/Jan 1, Biratnagar lai pradesh 1 k...","Protest / Demonstration, News Media, Incident,",26.484876,87.289331,524 1 02 05 2 001,Municipality/VDC,NaN,...,0.0,0,0,0,0,NaN,NaN,Checked by Admin,NaN,NaN
17479,News: Protest continues in Dhankuta | à¤§à¤¨à¤...,2017-12-31,"Dhankuta, Dhankuta, KOSHI, Eastern Development...","Alert: Dhankuta/Dec 31, Dhankuta lai pradesh 1...","Protest / Demonstration, News Media, Incident,",27.032196,87.318782,524 1 02 07 3 001,Municipality/VDC,NaN,...,0.0,0,0,0,0,0,NaN,Checked by Admin,NaN,NaN
17477,News: Woman kills husband by hitting with a wo...,2017-12-31,"Bhimsen-8, Gorkha, GANDAKI, Western Developmen...","Alert: Gorkha/Dec 31, Gharelu jhagada ko kram ...","Killings, DEATHS, Trusted Reports, Domestic Vi...",27.982000,84.751000,524 3 07 36 5 004 5,Ward,NaN,...,0.0,0,0,0,0,NaN,NaN,Checked by Admin,NaN,NaN


In [29]:
diffdropped.head()

,INCIDENT TITLE,INCIDENT DATE,LOCATION,DESCRIPTION,CATEGORY,LATITUDE,LONGITUDE,HLCIT CODE,LOCATION ACCURACY,Development Region,...,Female Abducted,Youth -25 years or below- abducted,Total Buildings,Severe Building Damage,Total Vehicles,Severe Vehicle Damage,Election-related specific cause primary,Election-related specific cause secondary,Verification Feedback,VERIFIED
INCIDENT NUMBER,,,,,,,,,,,,,,,,,,,,,
17599,Sajha kura: A man arrested with homemade gun |...,2017-12-28,"Makwanpur, NARAYANI, Central Development Regio...","Alert: Makwanpur/Dec 8, Bharuwa banduk sahit G...","Small Arms, Nepal Police (responding), ARREST,...",27.407938,85.117914,524 2 06 31,District,Central Development Region,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
17593,News:Engineer yadav arrested | इन्जिनीर यादव ...,2017-12-18,"Lahan, Siraha",Alert:Siraha/Dec 17; Siraha ka ek jana enginee...,"ARREST, Corruption / Fraud, News Media, CIAA, ...",26.713281,86.470451,524 1 03 16 3 002,Municipality/VDC,Eastern Development Region,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
17591,News: Woman rescued from being trafficked | बे...,2017-12-18,"Mechinagar, Jhapa, MECHI, Eastern Development ...","Alert: Parsa/Dec 18, Ramro rojgari ko lagi Del...","Human Trafficking, Nepal Police (responding), ...",26.770943,88.168507,524 1 01 04 5 018,Municipality/VDC,Eastern Development Region,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
17582,News: Parent mistreated by a teacher | शिक्षकद...,2017-12-14,"Mahagadhimai, Bara, NARAYANI, Central Developm...","Alert: Bara/Dec 15, Mahagadhimai nagarpalika-1...","Threat, News Media, Incident,",27.021855,85.059926,524 2 06 33 5 029,Municipality/VDC,Central Development Region,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0
17550,Janakpur Today: A man arrested with bullets | ...,2017-12-24,"Gaur-2, Rautahat, NARAYANI, Central Developmen...","Alert: Rautahat/Dec 26, 1 than goli ra pechkas...","Small Arms, Nepal Police (responding), ARREST,...",26.761275,85.266560,524 2 06 32 3 001 5,Ward,Central Development Region,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [30]:
#lets see if we can replicate diffdropped with our own function. Duplcates can by flagged
#by finding any records with both matching date and place. HLCIT codes store places in a hierarchy.

#maybe we can count duplicate place codes after grouping by date. 
dupsgroup = full17df.groupby(['INCIDENT DATE','HLCIT CODE']).count()

#create a boolean array of HLCIT codes in which their are more than one incident on a given incident date
dupsflag = dupsgroup['INCIDENT TITLE'] > 1 #we can use incident title here because we used count and not count unqique
dupsflag.name = 'DUPLICATE FLAG' # make dups flag into a multi-indexed boolean

#do a join with the original list on the dates and codes. This adds a column called duplicate flag in the original dataset
dupsjoin = full17df.join(dupsflag, on=('INCIDENT DATE','HLCIT CODE'), how='left') 

#create a dataframe of only the duplicates so we can look at it. 
dupsdf = dupsjoin.loc[dupsjoin['DUPLICATE FLAG'] == True]

#Drop rows where HLCIT code is too short to be very local. 
for num in dupsdf.index:
    if len(dupsdf.loc[num]['HLCIT CODE']) <= 11:
        dupsdf = dupsdf.drop(num)

print('Dupsdf shape: ',dupsdf.shape)
dupsdf.head()


Dupsdf shape:  (249, 69)


,INCIDENT TITLE,INCIDENT DATE,LOCATION,DESCRIPTION,CATEGORY,LATITUDE,LONGITUDE,HLCIT CODE,LOCATION ACCURACY,Development Region,...,Total Buildings,Severe Building Damage,Total Vehicles,Severe Vehicle Damage,Election-related specific cause primary,Election-related specific cause secondary,Verification Feedback,VERIFIED,difflist,DUPLICATE FLAG
INCIDENT NUMBER,,,,,,,,,,,,,,,,,,,,,
17685,Newspaper: Bank section of transportation mana...,2017-12-30,"Janakpur, Dhanusa, JANAKPUR, Central Developme...","Alert: Dhanusha/Dec 30, Yatayat byawastha kary...","News Media, Incident, Announcements / Demands,...",26.730245,85.927913,524 2 04 17 3 001,Municipality/VDC,Central Development Region,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0,False,True
17684,Janakpur Express: Madhesh movement victims pro...,2017-12-30,"Janakpur, Dhanusa, JANAKPUR, Central Developme...","Alert: Dhanusha/Dec 30, Tesro madhesh andolan ...","LOCAL COMMUNITIES, Protest / Demonstration, Ne...",26.727996,85.927458,524 2 04 17 3 001,Municipality/VDC,Central Development Region,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0,False,True
17600,Dhangadhi Post: FSU and employees padlocked Da...,2017-12-21,"Amargadhi, Dadeldhura, MAHAKALI, Far-Western D...","Alert: Dadeldhura/Dec 21, Campus pramukh ko ra...","STUDENT UNIONS AFFILIATED TO POLITICAL PARTY, ...",29.302073,80.593389,524 5 14 73 3 001,Municipality/VDC,Far-Western Development Region,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0,False,True
17472,NayaPatrika: Maoist Center leaders misbehaved ...,2017-12-29,"Kathmandu, Kathmandu, BAGMATI, Central Develop...","Alert: Kathmandu/Dec 30, Samanupatik tarfa pra...","CPN (Maoist Center), Assault / Violent Acts, N...",27.713335,85.323148,524 2 05 27 1 001,Municipality/VDC,Central Development Region,...,0.0,0.0,0.0,0.0,Intraparty competition,NaN,NaN,0,False,True
17462,News: NSU takes to streets demanding relocatio...,2017-12-29,"Kathmandu, Kathmandu, BAGMATI, Central Develop...","Alert: Kathmandu/Dec 29, Gyaas udhyog basti ba...","News Media, Incident, Announcements / Demands,",27.713335,85.323148,524 2 05 27 1 001,Municipality/VDC,Central Development Region,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0,False,True


In [31]:
#There are apparently 249 incidents that happened on the same date in the same locale 
#Let's view them. 
def print_front(df): #with a function so we don't have to copy paste
    df = df.sort_values(by='INCIDENT DATE')
    print(df.loc[:][['HLCIT CODE','INCIDENT DATE','INCIDENT TITLE']])

print_front(dupsdf) #obviously some are duplicates, see incident 17437. 

                           HLCIT CODE INCIDENT DATE  \
INCIDENT NUMBER                                       
12819               524 1 03 14 3 001    2017-01-04   
12822               524 1 03 14 3 001    2017-01-04   
13443               524 5 14 72 3 001    2017-01-16   
13447               524 5 14 72 3 001    2017-01-16   
13081               524 4 11 57 3 001    2017-01-22   
13014               524 4 11 57 3 001    2017-01-22   
13018               524 2 05 27 1 001    2017-01-23   
13020               524 2 05 27 1 001    2017-01-23   
13042               524 2 05 27 1 001    2017-01-25   
13064               524 2 05 27 1 001    2017-01-25   
13077               524 1 02 05 2 001    2017-01-25   
13127               524 2 04 17 3 001    2017-01-25   
13131               524 2 04 17 3 001    2017-01-25   
13439               524 1 02 05 2 001    2017-01-25   
13070               524 2 06 33 3 001    2017-01-26   
13084               524 2 06 33 3 001    2017-01-26   
13202     

In [32]:

#Compare shapes of diffs of dropped to this data set.
print('diffdropped:', diffdropped.shape)
imissed = makediff(diffdropped, dupsdf) #This shoud be the dulicates I missed
print('imissed:', imissed.shape)
theymissed = makediff(dupsdf, diffdropped)#this should be what they didn't remove but are next to each other on the same date
print('theymissed:', theymissed.shape)

#doesn't look like they intersect very much, maybe save as a file and go over manually, print side by side? 

diffdropped: (573, 67)
imissed: (549, 67)
theymissed: (225, 68)


/home/nhorning/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [33]:
# Function to find duplicates. Prints duplicates found in a given column, returns a df of them multi indexed. 
def find_duplicate(df,column_name,dd):
    multidups = df
    multidups['INCIDENT NUMBER'] = multidups.index.tolist()
    multidups = multidups.set_index(['INCIDENT DATE','HLCIT CODE','INCIDENT NUMBER'])
    multidups = multidups.sort_index()
   
    #check for duplicates in column blocks by date and print
    dup_count = 0
    dup_dict = {}
    for date in set(df['INCIDENT DATE']):  
        columnblock = multidups.loc[date][column_name]
        print_items = []
        for item in columnblock:
            if type(item) is str and columnblock.tolist().count(item) > 1:
                print_items.append(item)
                dup_count += 1
        print_items = list(set(print_items))
        if print_items != []:
            item_list = []
            print('\nDuplicate', column_name,'found for:',date)
            for print_item in print_items:
                rows = columnblock[columnblock == print_item]
                print(rows)
                item_list.append(rows.index.tolist())
            dup_dict[date] = item_list
    print('\nTotal Duplicate',column_name, 'found:', dup_count,'\n')
    if dd == True: 
        dup_discriptions(dup_dict, df)
    return dup_dict
    #Also return a dictionary of duplicate incident mumbers, for printing. 

def dup_discriptions(dup_dict,df): # Function to view discriptions of duplicates. Need to generalize and use df instead of dict
    for date in dup_dict.keys():
        print('Possible duplicates for:', date)
        for item in dup_dict[date]:
            for item in item:
                code = item[0]
                num = item[1]   
                print('TITLE:', df.loc[num]['INCIDENT TITLE'])
                print('lat/lon:', df.loc[num]['LATITUDE'], df.loc[num]['LONGITUDE'])
                print('HLCIT CODE:', code) 
                print(df.loc[num][['District','Municipality/VDC','Ward','Municipality/VDC(New)','Ward(New)']],'\n')
                print('Description', df.loc[num]['DESCRIPTION'],'\n')


#Check for duplicate titles on the same date in other data sets 
dup_dict_test = find_duplicate(dupsdf,'INCIDENT TITLE',dd=True)
diffdroppeddt = find_duplicate(diffdropped,'INCIDENT TITLE',dd=True)



Duplicate INCIDENT TITLE found for: 2017-12-22 00:00:00
HLCIT CODE         INCIDENT NUMBER
524 1 02 06 5 007  17436              News: E-rickshaw operators and police clash in...
                   17437              News: E-rickshaw operators and police clash in...
Name: INCIDENT TITLE, dtype: object

Total Duplicate INCIDENT TITLE found: 2 

Possible duplicates for: 2017-12-22 00:00:00
TITLE: News: E-rickshaw operators and police clash in Itahari | रिक्सा चालक र प्रहरीबीच झडप ।
lat/lon: 26.6882 87.263291
HLCIT CODE: 524 1 02 06 5 007
District                    Sunsari
Municipality/VDC         Hanshposha
Ward                            NaN
Municipality/VDC(New)       Itahari
Ward(New)                       NaN
Name: 17436, dtype: object 

Description Alert: Sunsari/Dec 22, Rajmarga ma City Safari gudauna pratibandha lagayeko birodh ma utriyeka chalak ra prahari bich jhadap. 

A clash erupted between e-rickshaw operators and police in Itahari Municipality. The clash took place when e

/home/nhorning/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.



Total Duplicate INCIDENT TITLE found: 2 

Possible duplicates for: 2017-01-13 00:00:00
TITLE: INSEC: Accused bigamous arrested | बहुविवाह घटनाको  आरोपि  गिरफ्तार ।
lat/lon: 27.13 87.816
HLCIT CODE: 524 1 01 02 5 020 1
District                 Panchthar
Municipality/VDC           Ranitar
Ward                             1
Municipality/VDC(New)       Phidim
Ward(New)                        7
Name: 12939, dtype: object 

Description Alert: Panchthar/Jan 13, Bahu-bibaha gareko arop Ranitaar gabisa-1 ka ek dampati prahari dwara pakrau. 

Police have arrested Rodan Kumar Limbu,33 of Ranitaar VDC-1 and Sabita Newang Limbu of the same VDC on January 13 on charge of polygamy.

Yonghang Limbu, 31 had lodged a complaint against both of them at DPO on January 10 for polygamy. The accused were arrested by the police based on the complaint lodged by the victim according to DSP Nakul Pokhrel of DPO.

Both the accused had married on November 10 and complaint was lodged demanding for legal treatment. 

We can see that two duplicate titles were found on the same date in the same place in the Full 2017 export. 
However, in the file of duplicates that were already dropped we can see two identical titles on the same date, 
but a different location. This could mean we cant count on duplcates to always be listed in the same location.

However, the discription of the tuplicate titles in different locations shows they are completely different incidents,
while the duplicates titles found in the same location have exactly the same description. Let's make a note to delete them.

## DROP incident no. 17437 out of final data
Lets feed the other datasets through. To see what else we can find.


In [34]:
print(statsdf,'\n')
print(statsdf.loc['full17df'].to_frame().T,'|----------------------------------\n')
full17dupsbytitle = find_duplicate(full17df,'INCIDENT TITLE',dd=True)

                  shape                                 comments
Dataframe                                                       
full17df     (4115, 68)                 The original 2017 export
ND17df       (3555, 70)                       No duplicates file
diff17df       (13, 69)         Added to make no duplicates file
diffdropped   (573, 67)  The duplicates - Removed to make ND17df 

               shape                  comments
full17df  (4115, 68)  The original 2017 export |----------------------------------


Duplicate INCIDENT TITLE found for: 2017-12-04 00:00:00
HLCIT CODE            INCIDENT NUMBER
524 2 05 27 1 001 2   17195              News: Explosive disposed | विष्फोटक पदार्थ डिस...
524 3 09 48 3 002 14  17207              News: Explosive disposed | विष्फोटक पदार्थ डिस...
Name: INCIDENT TITLE, dtype: object

Duplicate INCIDENT TITLE found for: 2017-12-22 00:00:00
HLCIT CODE         INCIDENT NUMBER
524 1 02 06 5 007  17436              News: E-rickshaw operators and pol

/home/nhorning/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


 INCIDENT TITLE found for: 2017-10-07 00:00:00
HLCIT CODE           INCIDENT NUMBER
524 3 09 51 5 079 6  16359              INSEC: Rape accused arrested | बलात्कार घटनाका...
524 5 13 68 5 013 8  16364              INSEC: Rape accused arrested | बलात्कार घटनाका...
Name: INCIDENT TITLE, dtype: object

Duplicate INCIDENT TITLE found for: 2017-12-06 00:00:00
HLCIT CODE           INCIDENT NUMBER
524 3 07 41 3 001 7  17251              News: Explosives defused | विष्फोटक पदार्थ डिस...
524 3 07 41 5 022 6  17253              News: Explosives defused | विष्फोटक पदार्थ डिस...
Name: INCIDENT TITLE, dtype: object

Duplicate INCIDENT TITLE found for: 2017-06-08 00:00:00
HLCIT CODE           INCIDENT NUMBER
524 1 02 05 5 051    14949              INSEC: Rape accused arrested | बलात्कार घटनाका...
524 2 06 31 5 008 4  14961              INSEC: Rape accused arrested | बलात्कार घटनाका...
524 3 07 36 5 016 2  14966              INSEC: Rape accused arrested | बलात्कार घटनाका...
Name: INCIDENT TITLE, dtyp


Description Alert: Bajura/Oct 8, 15 barshiya balika lai balatkaar gareko aarop ma Budhinanda-3 ka ek kishor pakrau, 2 balak farar.

Police have arrested 18 years old Roshan B.K of Budhi Nanda Municiality-3 on October 7 on charge of raping 15 years old girl. Police said that other two 15 year old boys are at large.

The victim’s party had lodged a complaint at police office on October 6 and one of the accused was arrested for the investigation according to SI Krishna Khadka of District Police Office.

The victim underwent a health checkup at District Hospital of Bajura on October 7.

१५ वर्षीया बालिकालाई बलात्कार गरेको आरोपमा प्रहरीले बुढीनन्दा नगरपालिका-३ का १८ वर्षीय रोशन बिकलाई असोज २१ गते गरिफ्तार गरेको र १५ वर्षीय दुई जना बालक फरार रहेको जनाइएको छ ।

पीडित बालिकालाई आरोपित त्रयले बलत्कार गरेको पीडित पक्षले असोज २० गते उजुरी दिएपछि उजुरीका आधारमा अनुसन्धानका लागि आरोपितलाई गिरफ्तार गरिएको जिल्ला प्रहरी कार्यालयका सई कृष्ण खड्काले बताए । पीडित बालिकाको स्वास्थ्य परीक्षण असोज २१ गते 

## 1 Genuine duplicate pair found for the original 2017 export (full17df)

21 duplicate incident titles were found for the same date in the full 2017 export. However, checking the descriptions reveals that they do not refer to the same incident on the ground. 1 definate duplicate pair is shown on 2017-12-22 (17436, 17437), as the discription matches completely. Was it caught by the team who made the no duplicates file?

Another pair on 17-12-06 (17251, 17253) could consists of two bombs in places apparently next to each other, that warrened further investigation. Nepal's transition to federalism has created a new naming scheme, listing the new wards next to their descriptions shows that they are indeed mapped in seperate appropriate places in the new naming scheme. Googling the gps coordinates shows they are about 30km apart, and can't be ruled out as seperate incidents.




In [35]:
#print(full17df.columns)
print(statsdf.loc['ND17df'].to_frame().T,'|----------------------------------\n')
ND17dupdt = find_duplicate(ND17df,'INCIDENT TITLE',dd=True)

             shape            comments
ND17df  (3555, 70)  No duplicates file |----------------------------------


Duplicate INCIDENT TITLE found for: 2017-12-04 00:00:00
HLCIT CODE            INCIDENT NUMBER
524 2 05 27 1 001 2   17195              News: Explosive disposed | à¤µà¤¿à¤·à¥à¤«à¥‹à...
524 3 09 48 3 002 14  17207              News: Explosive disposed | à¤µà¤¿à¤·à¥à¤«à¥‹à...
Name: INCIDENT TITLE, dtype: object

Duplicate INCIDENT TITLE found for: 2017-12-22 00:00:00
HLCIT CODE         INCIDENT NUMBER
524 1 02 06 5 007  17436              News: E-rickshaw operators and police clash in...
                   17437              News: E-rickshaw operators and police clash in...
Name: INCIDENT TITLE, dtype: object

Duplicate INCIDENT TITLE found for: 2017-08-15 00:00:00
HLCIT CODE           INCIDENT NUMBER
524 4 10 56 5 036 8  15885              INSEC: Arrested on charge of rape | à¤¬à¤²à¤¾à...
524 5 14 74 3 001 1  15883              INSEC: Arrested on charge of rape | à¤¬à¤²à¤


Possible duplicates for: 2017-08-15 00:00:00
TITLE: INSEC: Arrested on charge of rape | à¤¬à¤²à¤¾à¤¤à¥à¤•à¤¾à¤° à¤˜à¤Ÿà¤¨à¤¾à¤•à¤¾ à¤†à¤°à¥‹à¤ªà¤¿à¤¤ à¤—à¤¿à¤°à¤«à¥à¤¤à¤¾à¤° à¥¤
lat/lon: 28.173245 82.094435
HLCIT CODE: 524 4 10 56 5 036 8
District                       Dang
Municipality/VDC         Purandhara
Ward                              8
Municipality/VDC(New)         Babai
Ward(New)                         5
Name: 15885, dtype: object 

Description Alert: Dang/Aug 16, Ek yuwati lai balatkaar gareko arop ma prahari dwara Sombar Babai-5 ka ek yuwak pakrau.

Police have arrested 26 years old Indra Bahadur Oli of Babai village council-5 on charge of raping 21 years old woman.

The victimâ€™s party had lodged a complaint at District Police Office on August 15 on charge of raping victim on August 14.

The victim underwent a health checkup at Rapti Zonal Hospital, Tulsipur on August 15 according to the police.

à¥¨à¥§ à¤µà¤°à¥à¤·à¥€à¤¯à¤¾ à¤®à¤¹à¤¿à¤²à¤¾à¤²à¤¾à¤ˆ à¤¬à¤²à¤¾à¤¤à¥à¤•

## No other duplicates found,  but the E-rickshaw operator clash on 2017-12-22 (17436,17437) is still present. 

Excel has garbled the nepali text. We'll just have to index one by the other when we get there.
Also remember to run the VDC columns through the duplicate function to get duplicates by location. Search for duplicates within a date range using a date delta?  
Idea!!!  check if any name is included in the other one! just check if num + years old patern matches

This means that most of the duplicates the team dropped had to be mostly from manual checking, and they missed an obvious one. Let's see if we get any more identified duplicates by finding duplicate locations by date. 

In [36]:
#check for duplicates by VDC in original 2017 export
full17dupsbyvdc = find_duplicate(full17df,'Municipality/VDC',dd=False)


Duplicate Municipality/VDC found for: 2017-04-19 00:00:00


/home/nhorning/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


HLCIT CODE         INCIDENT NUMBER
524 1 03 16 3 002  14199              Lahan N.P.
                   14230              Lahan N.P.
                   14237              Lahan N.P.
Name: Municipality/VDC, dtype: object
HLCIT CODE         INCIDENT NUMBER
524 1 03 15 3 001  14215              Rajbiraj N.P.
                   14438              Rajbiraj N.P.
                   14442              Rajbiraj N.P.
Name: Municipality/VDC, dtype: object

Duplicate Municipality/VDC found for: 2017-05-19 00:00:00
HLCIT CODE           INCIDENT NUMBER
524 2 05 29 5 017    14731              Syafru
524 2 05 29 5 017 8  14730              Syafru
Name: Municipality/VDC, dtype: object

Duplicate Municipality/VDC found for: 2017-08-28 00:00:00
HLCIT CODE            INCIDENT NUMBER
524 2 05 27 1 001 13  15973              Kathmandu M.N.P.
524 2 05 27 1 001 3   15979              Kathmandu M.N.P.
Name: Municipality/VDC, dtype: object

Duplicate Municipality/VDC found for: 2017-09-12 00:00:00
HLCIT CODE   

Name: Municipality/VDC, dtype: object
HLCIT CODE            INCIDENT NUMBER
524 5 14 72 3 001     16143              Mahendra Nagar N.P.
524 5 14 72 3 001 11  16092              Mahendra Nagar N.P.
Name: Municipality/VDC, dtype: object

Duplicate Municipality/VDC found for: 2017-01-16 00:00:00
HLCIT CODE         INCIDENT NUMBER
524 5 14 72 3 001  13443              Mahendra Nagar N.P.
                   13447              Mahendra Nagar N.P.
Name: Municipality/VDC, dtype: object

Duplicate Municipality/VDC found for: 2017-12-07 00:00:00
HLCIT CODE           INCIDENT NUMBER
524 4 10 56 5 030    17295              PawanNagar
524 4 10 56 5 030 1  17247              PawanNagar
Name: Municipality/VDC, dtype: object
HLCIT CODE            INCIDENT NUMBER
524 1 02 06 3 001 16  17245              Dharan N.P.
524 1 02 06 3 001 9   17329              Dharan N.P.
Name: Municipality/VDC, dtype: object
HLCIT CODE           INCIDENT NUMBER
524 2 04 17 3 001    17651              Janakpur N.P.
524 2 0

Duplicate Municipality/VDC found for: 2017-03-02 00:00:00
HLCIT CODE            INCIDENT NUMBER
524 2 05 27 1 001 10  13487              Kathmandu M.N.P.
524 2 05 27 1 001 31  13495              Kathmandu M.N.P.
Name: Municipality/VDC, dtype: object

Duplicate Municipality/VDC found for: 2017-04-18 00:00:00
HLCIT CODE         INCIDENT NUMBER
524 2 06 33 5 096  14302              JitpurBhawanipur
                   14668              JitpurBhawanipur
Name: Municipality/VDC, dtype: object
HLCIT CODE           INCIDENT NUMBER
524 1 03 16 5 022    14200              Dhangadi
524 1 03 16 5 022 5  14178              Dhangadi
Name: Municipality/VDC, dtype: object

Duplicate Municipality/VDC found for: 2017-06-07 00:00:00
HLCIT CODE         INCIDENT NUMBER
524 4 11 57 3 001  14913              Nepalgunj N.P.
                   14943              Nepalgunj N.P.
Name: Municipality/VDC, dtype: object
HLCIT CODE           INCIDENT NUMBER
524 1 03 15 5 005 7  14945              Dhodhanpur
         

Duplicate Municipality/VDC found for: 2017-04-26 00:00:00
HLCIT CODE            INCIDENT NUMBER
524 2 05 27 1 001     14296              Kathmandu M.N.P.
524 2 05 27 1 001 31  14294              Kathmandu M.N.P.
Name: Municipality/VDC, dtype: object

Duplicate Municipality/VDC found for: 2017-08-17 00:00:00
HLCIT CODE           INCIDENT NUMBER
524 2 05 27 1 001 7  15897              Kathmandu M.N.P.
524 2 05 27 1 001 9  15887              Kathmandu M.N.P.
Name: Municipality/VDC, dtype: object

Duplicate Municipality/VDC found for: 2017-01-26 00:00:00
HLCIT CODE         INCIDENT NUMBER
524 2 06 33 3 001  13070              Kalaiya N.P.
                   13084              Kalaiya N.P.
Name: Municipality/VDC, dtype: object

Duplicate Municipality/VDC found for: 2017-02-23 00:00:00
HLCIT CODE            INCIDENT NUMBER
524 2 05 27 1 001     13396              Kathmandu M.N.P.
524 2 05 27 1 001 17  13425              Kathmandu M.N.P.
Name: Municipality/VDC, dtype: object
HLCIT CODE       

Duplicate Municipality/VDC found for: 2017-03-09 00:00:00
HLCIT CODE         INCIDENT NUMBER
524 1 03 15 3 001  13595              Rajbiraj N.P.
                   13599              Rajbiraj N.P.
Name: Municipality/VDC, dtype: object

Duplicate Municipality/VDC found for: 2017-03-22 00:00:00
HLCIT CODE         INCIDENT NUMBER
524 1 02 05 2 001  14085              Biratnagar U.M.N.P.
                   14088              Biratnagar U.M.N.P.
Name: Municipality/VDC, dtype: object

Duplicate Municipality/VDC found for: 2017-05-21 00:00:00
HLCIT CODE            INCIDENT NUMBER
524 2 05 27 1 001     14742              Kathmandu M.N.P.
524 2 05 27 1 001 14  14756              Kathmandu M.N.P.
Name: Municipality/VDC, dtype: object

Duplicate Municipality/VDC found for: 2017-02-26 00:00:00
HLCIT CODE         INCIDENT NUMBER
524 2 05 27 1 001  13412              Kathmandu M.N.P.
                   13416              Kathmandu M.N.P.
Name: Municipality/VDC, dtype: object

Duplicate Municipality/

In [37]:
#Check in no duplicate file for dups
ND17dupsdfbyvdc = find_duplicate(ND17df,'Municipality/VDC',dd=False)


Duplicate Municipality/VDC found for: 2017-04-19 00:00:00
HLCIT CODE         INCIDENT NUMBER
524 1 03 16 3 002  14199              Lahan N.P.
                   14230              Lahan N.P.
                   14237              Lahan N.P.
Name: Municipality/VDC, dtype: object
HLCIT CODE         INCIDENT NUMBER
524 1 03 15 3 001  14215              Rajbiraj N.P.
                   14438              Rajbiraj N.P.
                   14442              Rajbiraj N.P.
Name: Municipality/VDC, dtype: object

Duplicate Municipality/VDC found for: 2017-05-19 00:00:00
HLCIT CODE           INCIDENT NUMBER
524 2 05 29 5 017    14731              Syafru
524 2 05 29 5 017 8  14730              Syafru
Name: Municipality/VDC, dtype: object

Duplicate Municipality/VDC found for: 2017-09-12 00:00:00
HLCIT CODE           INCIDENT NUMBER
524 3 07 39 2 001    16250              Pokhara U.M.N.P.
524 3 07 39 2 001 9  16133              Pokhara U.M.N.P.
Name: Municipality/VDC, dtype: object

Duplicate Munic

Duplicate Municipality/VDC found for: 2017-02-22 00:00:00
HLCIT CODE         INCIDENT NUMBER
524 1 03 15 3 001  13373              Rajbiraj N.P.
                   13374              Rajbiraj N.P.
                   13382              Rajbiraj N.P.
Name: Municipality/VDC, dtype: object

Duplicate Municipality/VDC found for: 2017-08-15 00:00:00
HLCIT CODE         INCIDENT NUMBER
524 3 07 39 2 001  15867              Pokhara U.M.N.P.
                   15869              Pokhara U.M.N.P.
Name: Municipality/VDC, dtype: object

Duplicate Municipality/VDC found for: 2017-01-11 00:00:00
HLCIT CODE            INCIDENT NUMBER
524 3 07 39 2 001     12944              Pokhara U.M.N.P.
524 3 07 39 2 001 17  12916              Pokhara U.M.N.P.
Name: Municipality/VDC, dtype: object

Duplicate Municipality/VDC found for: 2017-10-09 00:00:00
HLCIT CODE            INCIDENT NUMBER
524 2 05 27 1 001     16395              Kathmandu M.N.P.
524 2 05 27 1 001 1   16396              Kathmandu M.N.P.
524 2 0

HLCIT CODE            INCIDENT NUMBER
524 2 05 27 1 001     16599              Kathmandu M.N.P.
524 2 05 27 1 001 15  16595              Kathmandu M.N.P.
Name: Municipality/VDC, dtype: object

Duplicate Municipality/VDC found for: 2017-12-29 00:00:00
HLCIT CODE            INCIDENT NUMBER
524 2 05 27 1 001     17462              Kathmandu M.N.P.
                      17472              Kathmandu M.N.P.
524 2 05 27 1 001 11  17473              Kathmandu M.N.P.
Name: Municipality/VDC, dtype: object

Duplicate Municipality/VDC found for: 2017-01-04 00:00:00
HLCIT CODE         INCIDENT NUMBER
524 1 03 14 3 001  12819              Triyuga N.P.
                   12822              Triyuga N.P.
Name: Municipality/VDC, dtype: object

Duplicate Municipality/VDC found for: 2017-12-21 00:00:00
HLCIT CODE         INCIDENT NUMBER
524 5 14 73 3 001  17394              Amargadhi N.P.
                   17600              Amargadhi N.P.
Name: Municipality/VDC, dtype: object

Duplicate Municipality/VDC

Duplicate Municipality/VDC found for: 2017-03-09 00:00:00
HLCIT CODE         INCIDENT NUMBER
524 1 03 15 3 001  13595              Rajbiraj N.P.
                   13599              Rajbiraj N.P.
Name: Municipality/VDC, dtype: object

Duplicate Municipality/VDC found for: 2017-08-07 00:00:00
HLCIT CODE            INCIDENT NUMBER
524 2 05 27 1 001     15792              Kathmandu M.N.P.
524 2 05 27 1 001 18  15802              Kathmandu M.N.P.
Name: Municipality/VDC, dtype: object

Duplicate Municipality/VDC found for: 2017-01-22 00:00:00
HLCIT CODE         INCIDENT NUMBER
524 4 11 57 3 001  13014              Nepalgunj N.P.
                   13081              Nepalgunj N.P.
Name: Municipality/VDC, dtype: object

Duplicate Municipality/VDC found for: 2017-03-22 00:00:00
HLCIT CODE         INCIDENT NUMBER
524 1 02 05 2 001  14085              Biratnagar U.M.N.P.
                   14088              Biratnagar U.M.N.P.
Name: Municipality/VDC, dtype: object

Duplicate Municipality/VDC 

## full17df: Total Duplicate Municipality/VDC found: 403
## ND17df: Total Duplicate Municipality/VDC found: 344 

There are about 60 duplicates dropped to make the ND17df file that have the same municipality VDC. It  therefore seems more effective to find duplicates by location than by title.

However, there are far too many listed here to check manually in a time effecent way. It's probably also better to be working with duplicate data frames than dictionaries and print-outs. The makediff function could be used to compare one to the other.

However,his is getting more into data enginering than data science need to cut it off and return later if necissary. 

In [38]:
ND17dupsdfbyvdc = find_duplicate(full17df,'DESCRIPTION',dd=True)

/home/nhorning/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.



Total Duplicate DESCRIPTION found: 0 



In [39]:
#Drop final list of duplicates from data. - including those that were dropped from the no duplicates file
dropdups= diffdropped.index.tolist()
dropdups.append(17437)
print(dropdups[-5:]) #veriy that was added.
fulldf.drop(dropdups, axis=0, inplace=True)

#check if that happened
for num in dropdups:
    for item in fulldf.index.tolist():
        if num == item:
            print(num, 'Not dropped!')

print (fulldf.index.tolist()[:5]) #check that these are similar lists

[12771, 12770, 12766, 12764, 17437]
[17685, 17684, 17675, 17670, 17668]


In [38]:
#append the missing incidents. 
fulldf.append(diff17df)
fulldf.head()

,INCIDENT TITLE,INCIDENT DATE,LOCATION,DESCRIPTION,CATEGORY,LATITUDE,LONGITUDE,HLCIT CODE,LOCATION ACCURACY,Development Region,...,Female Abducted,Youth -25 years or below- abducted,Total Buildings,Severe Building Damage,Total Vehicles,Severe Vehicle Damage,Election-related specific cause primary,Election-related specific cause secondary,Verification Feedback,VERIFIED
INCIDENT NUMBER,,,,,,,,,,,,,,,,,,,,,
17685,Newspaper: Bank section of transportation mana...,2017-12-30,"Janakpur, Dhanusa, JANAKPUR, Central Developme...","Alert: Dhanusha/Dec 30, Yatayat byawastha kary...","News Media, Incident, Announcements / Demands,...",26.730245,85.927913,524 2 04 17 3 001,Municipality/VDC,Central Development Region,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0
17684,Janakpur Express: Madhesh movement victims pro...,2017-12-30,"Janakpur, Dhanusa, JANAKPUR, Central Developme...","Alert: Dhanusha/Dec 30, Tesro madhesh andolan ...","LOCAL COMMUNITIES, Protest / Demonstration, Ne...",26.727996,85.927458,524 2 04 17 3 001,Municipality/VDC,Central Development Region,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0
17675,Janakpur Express: Nawarang hotel vandalized | ...,2017-12-27,"Janakpur, Dhanusa, JANAKPUR, Central Developme...","Alert: Dhanusha/Dec 27, Janakpur ko shiva chow...","Nepal Police (responding), News Media, Vandali...",26.735339,85.936133,524 2 04 17 3 001,Municipality/VDC,Central Development Region,...,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,0
17670,Janakpur Today: Students padlocked technical s...,2017-12-23,"Dhanusadham, Dhanusa, JANAKPUR, Central Develo...","Alert: Dhanusha/Dec 23, Niyamit pathanpathan k...","News Media, District Education Office / MoE, I...",26.819670,86.055004,524 2 04 17 5 050,Municipality/VDC,Central Development Region,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0
17668,Janakpur Today: Students protest in front of D...,2017-12-22,"Rajbiraj, Saptari, SAGARMATHA, Eastern Develop...","Alert: Saptari/Dec 22, Little angels boarding ...","Nepal Police (responding), Protest / Demonstra...",26.501006,86.772946,524 1 03 15 5 033,Municipality/VDC,Eastern Development Region,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0


## Save files to resume from another notebook when resuming project

In [39]:
#Save data
fulldfpath='Data/NM_2016_17clean.csv'
fulldf.to_csv(fulldfpath)

In [40]:
!ls Data/

1519759656_39328483.csv
2017 PMP Data Final No Duplicates.xlsx
District_Province_Population
Federal_VDC_Relation_web.xlsx
nepal_population_household_ward-2011.csv
NM_2016_17clean.csv
NM_2016_17export.csv
